In [1]:
# загружаем библиотеки
import pandas as pd
import xlrd
import re

In [2]:
# загружаем датасет
transgas_p = pd.read_excel('Трансгазы_flat_table.xlsx')
transgas_p.head()

,gas_type,date,tg_owner,tg_from,tg_to_grs,distance
0,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Екатеринбург""","ООО ""ГТ Сургут""","Асбест, ГРС-2",506.9
1,Транзитный газ,2018-01-01,"ООО ""Газпром трансгаз Екатеринбург""","ООО ""ГТ Сургут""","ООО ""ГТ Уфа""",379.0
2,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Екатеринбург""","ООО ""ГТ Сургут""",Бакал,508.7
3,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Екатеринбург""","ООО ""ГТ Сургут""",Береговой,366.9
4,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Екатеринбург""","ООО ""ГТ Сургут""",Богданович,513.4


In [3]:
# смотрим уникальные значения в столбце tg_owner
transgas_p['tg_owner'].unique()

array(['ООО "Газпром трансгаз Екатеринбург"',
       'ООО "Газпром трансгаз Казань"', 'ООО "Газпром трансгаз Москва"',
       'ООО "Газпром трансгаз Нижний Новгород"',
       'ООО "Газпром трансгаз Санкт-Петербург"',
       'ООО "Газпром трансгаз Саратов"', 'ООО "Газпром трансгаз Сургут"',
       'ООО "Газпром трансгаз Томск"', 'ООО "Газпром трансгаз Ухта"',
       'ООО "Газпром трансгаз Чайковский"',
       'ООО "Газпром трансгаз Югорск"'], dtype=object)

In [4]:
# смотрим уникальные значения в столбце tg_from
transgas_p['tg_from'].unique()

array(['ООО "ГТ Сургут"', 'ООО "ГТ Югорск"', 'ООО "ГТ Н. Новгород"',
       'ООО "ГТ Уфа"', 'ООО "ГТ Чайковский"', 'ООО "ГТ Самара"',
       'ООО "ГТ С-Петербург"', 'ООО "ГТ Казань"', 'ООО "ГТ Ухта"',
       'ООО "ГТ Москва"',
       'БП "622,5 км" (Локосово) (622,5 км газопровода Уренгой-Челябинск)',
       'Газопровод внешнего транспорта Вынгаяхинского месторождения, 0,35 км (ПАО "Газпром")',
       'Газопровод внешнего транспорта Комсомольского месторождения, 0 км (ОАО "НК "Янгпур")',
       '622 км Уренгой-Челябинск (ООО "Газ-Ойл трейдинг")',
       '0 км МГ Медвежье-Надым I,II (ПАО "Газпром")',
       'КС Надым (216,2 км газопровода Уренгой-Петровск)',
       '78 км МГ Уренгой-Надым I, II (ПАО "Газпром")',
       '0 км газопровода "Ямбуг-Тула" I и II очередь (КС Ямбургская) (ПАО "НОВАТЭК")'],
      dtype=object)

In [5]:
# смотрим уникальные значения в столбце tg_owner
transgas_p['tg_to_grs'].unique()

array(['Асбест, ГРС-2', 'ООО "ГТ Уфа"', 'Бакал', 'Береговой',
       'Богданович', 'Златоуст', 'Каменск-Уральский', 'Курган',
       'Полевской', 'Реж', 'Свердловск, ГРС-1', 'Свердловск, ГРС-3',
       'Свердловск, ГРС-4', 'Сухой Лог', 'Челябинск, ГРС-1',
       'Челябинск, ГРС-2', 'Челябинск, ГРС-3', 'Челябинск, ГРС-4',
       'Шадринск', 'Алапаевск*', 'Верх-Нейвинск', 'Верхняя Салда',
       'Кировград', 'Нижний Тагил, ГРС-1', 'Первоуральск, ГРС-1',
       'Казань №2', 'ООО "ГТ Самара"', 'Казань №5',
       'ООО "ГТ Н. Новгород"', 'Ашальчи*', 'Наб.Челнинская №3',
       'Нижнекамская №2', 'АГРС АО "Аммоний"*',
       'Елабуга №2 (Центральная)', 'Казань №1', 'Нижнекамская №3',
       'Белгород-1', 'ООО "ГТ Саратов"', 'Брянск-1', 'Воронеж-1',
       'Глинищево', 'Графовка', 'Железногорск', 'Изовер*', 'Косиново',
       'Кромы', 'Кроношпан*', 'КРП-10', 'Лебедянь', 'Лужки',
       'Майский (Белгород-2)', 'Мин. Удобрения (г.Воскресенск)',
       'Молотино', 'Новый Оскол', 'ОЭМК', 'Петровс

In [6]:
def transform (row):
    ''' Приводим названия tg_owner к стандартному виду вида "ГТ Екатеринбург"'''
    tr1 = row['tg_owner'].replace('Газпром трансгаз', 'ГТ')
    tr2 = tr1.replace('Нижний', 'Н.')
    tr3 = tr2.replace('Санкт', 'С')
    tr4 = tr3.split('"')[1]
    return tr4

In [7]:
# применяем функции transform
transgas_p['tg_owner'] = transgas_p.apply(transform, axis = 1)
transgas_p['tg_owner'].unique()

array(['ГТ Екатеринбург', 'ГТ Казань', 'ГТ Москва', 'ГТ Н. Новгород',
       'ГТ С-Петербург', 'ГТ Саратов', 'ГТ Сургут', 'ГТ Томск', 'ГТ Ухта',
       'ГТ Чайковский', 'ГТ Югорск'], dtype=object)

In [8]:
def transform_1 (row):
    ''' Приводим названия tg_from к стандартному виду вида "ГТ Екатеринбург"'''
    st = row['tg_from']
    result = re.search(r'ГТ', st)
    if result:
        s_index = result.start()
        return st[s_index:-1]
    else:
        return st

In [9]:
# применяем функции transform_1
transgas_p['tg_from'] = transgas_p.apply(transform_1, axis = 1)
transgas_p['tg_from'].unique()

array(['ГТ Сургут', 'ГТ Югорск', 'ГТ Н. Новгород', 'ГТ Уфа',
       'ГТ Чайковский', 'ГТ Самара', 'ГТ С-Петербург', 'ГТ Казань',
       'ГТ Ухта', 'ГТ Москва',
       'БП "622,5 км" (Локосово) (622,5 км газопровода Уренгой-Челябинск)',
       'Газопровод внешнего транспорта Вынгаяхинского месторождения, 0,35 км (ПАО "Газпром")',
       'Газопровод внешнего транспорта Комсомольского месторождения, 0 км (ОАО "НК "Янгпур")',
       '622 км Уренгой-Челябинск (ООО "Газ-Ойл трейдинг")',
       '0 км МГ Медвежье-Надым I,II (ПАО "Газпром")',
       'КС Надым (216,2 км газопровода Уренгой-Петровск)',
       '78 км МГ Уренгой-Надым I, II (ПАО "Газпром")',
       '0 км газопровода "Ямбуг-Тула" I и II очередь (КС Ямбургская) (ПАО "НОВАТЭК")'],
      dtype=object)

In [10]:
def transform_2 (row):
    ''' Приводим названия tg_to_grs к стандартному виду вида "ГТ Екатеринбург"'''
    st = row['tg_to_grs']
    result = re.search(r'ГТ', st)
    if result:
        s_index = result.start()
        return st[s_index:-1]
    else:
        return st

In [11]:
# применяем функции transform_2
transgas_p['tg_to_grs'] = transgas_p.apply(transform_2, axis = 1)
transgas_p['tg_to_grs'].unique()

array(['Асбест, ГРС-2', 'ГТ Уфа', 'Бакал', 'Береговой', 'Богданович',
       'Златоуст', 'Каменск-Уральский', 'Курган', 'Полевской', 'Реж',
       'Свердловск, ГРС-1', 'Свердловск, ГРС-3', 'Свердловск, ГРС-4',
       'Сухой Лог', 'Челябинск, ГРС-1', 'Челябинск, ГРС-2',
       'Челябинск, ГРС-3', 'Челябинск, ГРС-4', 'Шадринск', 'Алапаевск*',
       'Верх-Нейвинск', 'Верхняя Салда', 'Кировград',
       'Нижний Тагил, ГРС-1', 'Первоуральск, ГРС-1', 'Казань №2',
       'ГТ Самара', 'Казань №5', 'ГТ Н. Новгород', 'Ашальчи*',
       'Наб.Челнинская №3', 'Нижнекамская №2', 'АГРС АО "Аммоний"*',
       'Елабуга №2 (Центральная)', 'Казань №1', 'Нижнекамская №3',
       'Белгород-1', 'ГТ Саратов', 'Брянск-1', 'Воронеж-1', 'Глинищево',
       'Графовка', 'Железногорск', 'Изовер*', 'Косиново', 'Кромы',
       'Кроношпан*', 'КРП-10', 'Лебедянь', 'Лужки',
       'Майский (Белгород-2)', 'Мин. Удобрения (г.Воскресенск)',
       'Молотино', 'Новый Оскол', 'ОЭМК', 'Петровское (Видное)',
       'Рязань-1

In [12]:
# убираем не нужные столбцы
transgas = transgas_p.iloc[:, 2:]
transgas.head()

,tg_owner,tg_from,tg_to_grs,distance
0,ГТ Екатеринбург,ГТ Сургут,"Асбест, ГРС-2",506.9
1,ГТ Екатеринбург,ГТ Сургут,ГТ Уфа,379.0
2,ГТ Екатеринбург,ГТ Сургут,Бакал,508.7
3,ГТ Екатеринбург,ГТ Сургут,Береговой,366.9
4,ГТ Екатеринбург,ГТ Сургут,Богданович,513.4


In [13]:
# форируем список ГРС
grs = transgas_p[transgas_p['gas_type'] == 'Газ, для последующей реализации, переданный на ГРС Исполнителя и /или на границе газопроводов Исполнителя с газопроводами-отводами сторонней ГТО']
list_grs = list(grs['tg_to_grs'].unique())
print(list_grs)

['Асбест, ГРС-2', 'Бакал', 'Береговой', 'Богданович', 'Златоуст', 'Каменск-Уральский', 'Курган', 'Полевской', 'Реж', 'Свердловск, ГРС-1', 'Свердловск, ГРС-3', 'Свердловск, ГРС-4', 'Сухой Лог', 'Челябинск, ГРС-1', 'Челябинск, ГРС-2', 'Челябинск, ГРС-3', 'Челябинск, ГРС-4', 'Шадринск', 'Алапаевск*', 'Верх-Нейвинск', 'Верхняя Салда', 'Кировград', 'Нижний Тагил, ГРС-1', 'Первоуральск, ГРС-1', 'Казань №2', 'Казань №5', 'Ашальчи*', 'Наб.Челнинская №3', 'Нижнекамская №2', 'АГРС АО "Аммоний"*', 'Елабуга №2 (Центральная)', 'Казань №1', 'Нижнекамская №3', 'Белгород-1', 'Брянск-1', 'Воронеж-1', 'Глинищево', 'Графовка', 'Железногорск', 'Изовер*', 'Косиново', 'Кромы', 'Кроношпан*', 'КРП-10', 'Лебедянь', 'Лужки', 'Майский (Белгород-2)', 'Мин. Удобрения (г.Воскресенск)', 'Молотино', 'Новый Оскол', 'ОЭМК', 'Петровское (Видное)', 'Рязань-1', 'Рязань-3', 'Серпухов', 'Скопин', 'Тросна', 'Тульская', 'Шебекино', 'Конаково', 'Алексеевская', 'Володарск-1', 'Дубенки', 'Йошкар-Ола', 'Навашино', 'Пенза-2', 'Пен

In [14]:
# удаляем данные с неполным путем
list_grs.remove('Ашальчи*')
list_grs.remove('Наб.Челнинская №3')
list_grs.remove('Нижнекамская №2')
print(list_grs)

['Асбест, ГРС-2', 'Бакал', 'Береговой', 'Богданович', 'Златоуст', 'Каменск-Уральский', 'Курган', 'Полевской', 'Реж', 'Свердловск, ГРС-1', 'Свердловск, ГРС-3', 'Свердловск, ГРС-4', 'Сухой Лог', 'Челябинск, ГРС-1', 'Челябинск, ГРС-2', 'Челябинск, ГРС-3', 'Челябинск, ГРС-4', 'Шадринск', 'Алапаевск*', 'Верх-Нейвинск', 'Верхняя Салда', 'Кировград', 'Нижний Тагил, ГРС-1', 'Первоуральск, ГРС-1', 'Казань №2', 'Казань №5', 'АГРС АО "Аммоний"*', 'Елабуга №2 (Центральная)', 'Казань №1', 'Нижнекамская №3', 'Белгород-1', 'Брянск-1', 'Воронеж-1', 'Глинищево', 'Графовка', 'Железногорск', 'Изовер*', 'Косиново', 'Кромы', 'Кроношпан*', 'КРП-10', 'Лебедянь', 'Лужки', 'Майский (Белгород-2)', 'Мин. Удобрения (г.Воскресенск)', 'Молотино', 'Новый Оскол', 'ОЭМК', 'Петровское (Видное)', 'Рязань-1', 'Рязань-3', 'Серпухов', 'Скопин', 'Тросна', 'Тульская', 'Шебекино', 'Конаково', 'Алексеевская', 'Володарск-1', 'Дубенки', 'Йошкар-Ола', 'Навашино', 'Пенза-2', 'Пенза-4', 'Саранск-3', 'Чаадаевка', 'Владимир-3', 'Воро

In [15]:
def trek (grs_1):
    ''' Формироване наиболее короткого маршрута от ГРС до БП'''
    lst = [grs_1]
    line = transgas[(transgas['tg_to_grs'] == grs_1)].sort_values( by = 'distance', ascending = True)
    owner = line.iloc[0]['tg_owner']
    dist = line.iloc[0]['distance']
    tg_from = line.iloc[0]['tg_from']
    lst.append(owner)
    lst.append(dist)
    while owner != 'ГТ Югорск' and owner != 'ГТ Сургут':
        line = transgas[(transgas['tg_to_grs'] == owner) & (transgas['tg_owner'] == tg_from)].sort_values( by = 'distance', ascending = True)
        owner = line.iloc[0]['tg_owner']
        dist = line.iloc[0]['distance']
        tg_from = line.iloc[0]['tg_from']
        lst.append(owner)
        lst.append(dist)  
    if tg_from == 'КС Надым (216,2 км газопровода Уренгой-Петровск)':
        bp = 'Надым'    
    elif tg_from == 'БП "622,5 км" (Локосово) (622,5 км газопровода Уренгой-Челябинск)':
        bp = 'Локосово'
    lst.append(bp)  
    i = 4
    d_sum = lst[2]
    len_lst = len(lst)
    while i < len_lst:
        d_sum  = d_sum + lst[i]
        i = i + 2
    rd_sum = round(d_sum,2)
    lst.append(rd_sum)
    return lst

In [16]:
def main():
    ''' Формирование маршрутов для всех ГРС'''
    for i in list_grs:
        print(trek(i))

In [17]:
main()

['Асбест, ГРС-2', 'ГТ Екатеринбург', 506.9, 'ГТ Сургут', 863.5, 'Локосово', 1370.4]
['Бакал', 'ГТ Екатеринбург', 508.7, 'ГТ Сургут', 863.5, 'Локосово', 1372.2]
['Береговой', 'ГТ Екатеринбург', 366.9, 'ГТ Сургут', 863.5, 'Локосово', 1230.4]
['Богданович', 'ГТ Екатеринбург', 513.4, 'ГТ Сургут', 863.5, 'Локосово', 1376.9]
['Златоуст', 'ГТ Екатеринбург', 443.4, 'ГТ Сургут', 863.5, 'Локосово', 1306.9]
['Каменск-Уральский', 'ГТ Екатеринбург', 464.1, 'ГТ Сургут', 863.5, 'Локосово', 1327.6]
['Курган', 'ГТ Екатеринбург', 169.4, 'ГТ Сургут', 863.5, 'Локосово', 1032.9]
['Полевской', 'ГТ Екатеринбург', 452.4, 'ГТ Сургут', 863.5, 'Локосово', 1315.9]
['Реж', 'ГТ Екатеринбург', 552.0, 'ГТ Сургут', 863.5, 'Локосово', 1415.5]
['Свердловск, ГРС-1', 'ГТ Екатеринбург', 489.9, 'ГТ Сургут', 863.5, 'Локосово', 1353.4]
['Свердловск, ГРС-3', 'ГТ Екатеринбург', 440.8, 'ГТ Сургут', 863.5, 'Локосово', 1304.3]
['Свердловск, ГРС-4', 'ГТ Екатеринбург', 441.3, 'ГТ Сургут', 863.5, 'Локосово', 1304.8]
['Сухой Лог', 'ГТ

['Инжавино', 'ГТ Саратов', 163.72, 'ГТ Москва', 71.3, 'ГТ Н. Новгород', 532.35, 'ГТ Казань', 149.65, 'ГТ Чайковский', 653.62, 'ГТ Югорск', 1136.9, 'Надым', 2707.54]
['Нижний Ломов', 'ГТ Саратов', 134.447, 'ГТ Москва', 71.3, 'ГТ Н. Новгород', 532.35, 'ГТ Казань', 149.65, 'ГТ Чайковский', 653.62, 'ГТ Югорск', 1136.9, 'Надым', 2678.27]
['Тамбов, ГРС-2А', 'ГТ Саратов', 12.9, 'ГТ Москва', 71.3, 'ГТ Н. Новгород', 532.35, 'ГТ Казань', 149.65, 'ГТ Чайковский', 653.62, 'ГТ Югорск', 1136.9, 'Надым', 2556.72]
['ГРС Винзили', 'ГТ Сургут', 792.8, 'Локосово', 792.8]
['г.Омск ГPC-1А*', 'ГТ Томск', 218.4, 'ГТ Сургут', 1131.7, 'Локосово', 1350.1]
['ГPC-4 г. Омск', 'ГТ Томск', 251.9, 'ГТ Сургут', 1131.7, 'Локосово', 1383.6]
['ГPC-5 г. Oмск', 'ГТ Томск', 210.2, 'ГТ Сургут', 1131.7, 'Локосово', 1341.9]
['ГРС-2 г. Омск', 'ГТ Томск', 219.4, 'ГТ Сургут', 1131.7, 'Локосово', 1351.1]
['ГРС-21 "Москаленки"*', 'ГТ Томск', 198.8, 'ГТ Сургут', 1131.7, 'Локосово', 1330.5]
['ГРС-1 Северодвинск', 'ГТ Ухта', 1542.3, '

In [18]:
def main2():
    ''' Формирование маршрутов для всех ГРС'''
    res = []
    for i in list_grs:
        res.append(trek(i))
    return res

In [19]:
f_lst = main2()
print(f_lst)

[['Асбест, ГРС-2', 'ГТ Екатеринбург', 506.9, 'ГТ Сургут', 863.5, 'Локосово', 1370.4], ['Бакал', 'ГТ Екатеринбург', 508.7, 'ГТ Сургут', 863.5, 'Локосово', 1372.2], ['Береговой', 'ГТ Екатеринбург', 366.9, 'ГТ Сургут', 863.5, 'Локосово', 1230.4], ['Богданович', 'ГТ Екатеринбург', 513.4, 'ГТ Сургут', 863.5, 'Локосово', 1376.9], ['Златоуст', 'ГТ Екатеринбург', 443.4, 'ГТ Сургут', 863.5, 'Локосово', 1306.9], ['Каменск-Уральский', 'ГТ Екатеринбург', 464.1, 'ГТ Сургут', 863.5, 'Локосово', 1327.6], ['Курган', 'ГТ Екатеринбург', 169.4, 'ГТ Сургут', 863.5, 'Локосово', 1032.9], ['Полевской', 'ГТ Екатеринбург', 452.4, 'ГТ Сургут', 863.5, 'Локосово', 1315.9], ['Реж', 'ГТ Екатеринбург', 552.0, 'ГТ Сургут', 863.5, 'Локосово', 1415.5], ['Свердловск, ГРС-1', 'ГТ Екатеринбург', 489.9, 'ГТ Сургут', 863.5, 'Локосово', 1353.4], ['Свердловск, ГРС-3', 'ГТ Екатеринбург', 440.8, 'ГТ Сургут', 863.5, 'Локосово', 1304.3], ['Свердловск, ГРС-4', 'ГТ Екатеринбург', 441.3, 'ГТ Сургут', 863.5, 'Локосово', 1304.8], ['Су

In [20]:
with open ('trek.csv', 'w', encoding='utf-8') as f2w:
    for i in f_lst:
        string = ''
        for el in i:
            el_1 = str(el)
            el_k = el_1.replace(', ', '-')
            string += str(el_k)
            string += ', '
        f2w.write(f'{string}\n')